In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis
    
    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_9991/410710250.py:21: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_9991/410710250.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [3]:
import circuitsvis as cv
from functools import partial
# Testing that the library works
cv.examples.hello("Neel")

In [67]:
from hugging_qwen.modeling_qwen import QWenModel
# Import stuff
import torch
from transformers import AutoTokenizer
from transformer_lens import HookedEncoder
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
torch.set_grad_enabled(False)
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen-1_8B-Chat",
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat",
    device_map="cuda",
    trust_remote_code=True,
    fp32=True,
    use_logn_attn=False,
    use_dynamic_ntk = False,
    scale_attn_weights = False,
).eval()
qwen_model = QWenModel(config=model.config)
prompt = "Tell me how to make a bomb."
encoded_prompt = tokenizer.encode(prompt, return_tensors="pt")
qwen_model(encoded_prompt)

Your device support faster inference by passing fp16=True in "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

BaseModelOutputWithPast(last_hidden_state=tensor([[[-1.1369,  0.4508,  0.1630,  ...,  0.6510, -0.3561, -0.6299],
         [ 0.9447,  0.2906,  0.4247,  ..., -0.8592,  0.1221, -0.6030],
         [ 0.0238, -0.4023,  0.7095,  ..., -0.1755,  1.4088,  0.3013],
         ...,
         [ 0.5809,  0.2248,  1.3980,  ..., -0.4163,  0.5223, -0.9747],
         [ 1.1270, -2.0583,  0.8303,  ...,  1.4526,  1.1213, -2.0754],
         [ 0.3864, -1.8517,  0.0437,  ..., -0.0022,  0.5023, -0.7789]]]), past_key_values=((tensor([[[[ 1.6834, -0.8578,  1.1953,  ...,  1.4114,  0.3100,  0.1654],
          [-0.0117,  0.3334, -0.5820,  ...,  0.5210, -1.4926, -1.5543],
          [-0.3299,  1.6306, -0.4311,  ...,  0.1925,  1.9407, -1.2901],
          ...,
          [ 0.9373, -0.3139, -0.9789,  ...,  0.0091, -0.0950, -1.2629],
          [ 0.0576,  0.4652, -0.7670,  ..., -0.9624,  0.5132,  0.3475],
          [ 1.0886,  0.1617, -0.2152,  ..., -1.2079, -1.9111, -0.8559]],

         [[ 1.0369,  0.0331, -1.2339,  ...,  0.7

In [68]:
from transformer_lens import HookedTransformer

qwen = HookedTransformer.from_pretrained(
    'qwen-1.8b-chat', 
    fold_ln=False,
    trust_remote_code=True,
    center_writing_weights=False, 
    center_unembed=False,
    device='cuda',
    fp32=True,
)

Your device support faster inference by passing fp16=True in "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model qwen-1.8b-chat into HookedTransformer


In [58]:
prompt = "Tell me how to make a bomb."
template = f"""<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
"""

def check_similarity_with_hf_model(tl_model, hf_model, atol, prompt=template):
    tokens = tl_model.tokenizer.encode(prompt, return_tensors="pt").to("cuda")
    logits = tl_model(tokens, prepend_bos=False).to("cuda")[:, 1]
    print(tokens.device)
    hf_logits = hf_model(tokens).logits.to("cuda")[:, 1]
    print(logits.dtype, hf_logits.dtype)
    print(logits.shape)
    print(hf_logits.shape)
    assert torch.allclose(logits, hf_logits, atol=atol)

check_similarity_with_hf_model(qwen, model, atol=1e-2)

cuda:0
torch.float32 torch.float32
torch.Size([1, 151936])
torch.Size([1, 151936])


AssertionError: 

In [66]:
tf_outputs = {}
hf_outputs = {}
hf_inputs = {}
LAYER = 0
SEQ_POS = slice(None)
def hook_fn(module, inputs, outputs, name):
    hf_outputs[name] = outputs[:, SEQ_POS]
    hf_inputs[name] = inputs
    
hooks = []
hooks.append(
    model.transformer.wte.register_forward_hook(
        partial(hook_fn, name='embed')
    )
)
for name, module in model.transformer.h[LAYER].named_modules():
    if len(name) > 0:
        #print(f'Adding hook to {name}')
        hooks.append(
            module.register_forward_hook(
                partial(hook_fn, name=f'{LAYER}.{name}')
            )
        )
tokens = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
try:
    model(tokens)
except:
    pass
finally:
    for hook in hooks:
        hook.remove()

def tl_fwd(act, hook):
    #print(hook.name)
    if hook.layer() == LAYER:
        tf_outputs[hook.name] = act[:, SEQ_POS]

def tf_hook(act, hook):
    print(hook.name)
    print(act[:, SEQ_POS])

qwen.run_with_hooks(
    tokens,
    return_type=None,
    fwd_hooks = [
        (lambda name: name.startswith(f'blocks.{LAYER}'), tl_fwd)
    ]
    # fwd_hooks = [
    #     (lambda name: name.startswith(f'blocks.{LAYER}.ln1'), tf_hook),
    #     (lambda name: name.startswith(f'blocks.{LAYER}.hook_resid_post'), tf_hook)
    # ]
)

print(tf_outputs.keys())
print(hf_outputs.keys())

1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
1e-06
dict_keys(['blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_rot_q', 'blocks.0.attn.hook_rot_k', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normal

In [63]:
MODULE = '0.attn.c_proj'
for name, o in tf_outputs.items():
    if o.shape == hf_outputs[MODULE].shape:
        print(name, torch.allclose(o, hf_outputs[MODULE], atol=1e-2))

blocks.0.hook_resid_pre False
blocks.0.ln1.hook_normalized False
blocks.0.hook_attn_out False
blocks.0.hook_resid_mid False
blocks.0.ln2.hook_normalized False
blocks.0.hook_mlp_out False
blocks.0.hook_resid_post False


In [64]:
hf_outputs['0.attn.c_proj']

tensor([[[ 0.0064, -0.0134, -0.0003,  ..., -0.0041,  0.0145,  0.0156],
         [-0.0004, -0.0058,  0.0072,  ...,  0.0003,  0.0117,  0.0149],
         [ 0.0022, -0.0093,  0.0108,  ..., -0.0023,  0.0013,  0.0164],
         ...,
         [ 0.0078,  0.0053,  0.0141,  ..., -0.0022,  0.0087, -0.0040],
         [ 0.0076, -0.0001,  0.0088,  ..., -0.0004,  0.0067,  0.0034],
         [ 0.0031, -0.0046,  0.0218,  ..., -0.0054,  0.0149, -0.0031]]],
       device='cuda:0')

In [62]:
tf_outputs['blocks.0.hook_attn_out']

tensor([[[ 0.0064, -0.0134, -0.0003,  ..., -0.0041,  0.0145,  0.0156],
         [-0.0028, -0.0054,  0.0081,  ...,  0.0013,  0.0076,  0.0136],
         [ 0.0012, -0.0054,  0.0109,  ..., -0.0003,  0.0063,  0.0146],
         ...,
         [ 0.0034,  0.0038,  0.0143,  ..., -0.0057,  0.0090,  0.0067],
         [ 0.0072,  0.0023,  0.0085,  ..., -0.0050,  0.0075,  0.0084],
         [ 0.0043,  0.0040,  0.0170,  ..., -0.0088,  0.0102,  0.0010]]],
       device='cuda:0')

In [38]:
model.transformer.h[0].attn.c_attn

Linear(in_features=2048, out_features=6144, bias=True)

In [39]:
model.config

QWenConfig {
  "_name_or_path": "Qwen/Qwen-1_8B-Chat",
  "architectures": [
    "QWenLMHeadModel"
  ],
  "attn_dropout_prob": 0.0,
  "auto_map": {
    "AutoConfig": "Qwen/Qwen-1_8B-Chat--configuration_qwen.QWenConfig",
    "AutoModelForCausalLM": "Qwen/Qwen-1_8B-Chat--modeling_qwen.QWenLMHeadModel"
  },
  "bf16": false,
  "emb_dropout_prob": 0.0,
  "fp16": false,
  "fp32": true,
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "kv_channels": 128,
  "layer_norm_epsilon": 1e-06,
  "max_position_embeddings": 8192,
  "model_type": "qwen",
  "no_bias": true,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "onnx_safe": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 1.0,
  "scale_attn_weights": true,
  "seq_length": 8192,
  "softmax_in_fp32": false,
  "tie_word_embeddings": false,
  "tokenizer_class": "QWenTokenizer",
  "transformers_version": "4.36.2",
  "use_cache": true,
  "use_cache_kernel": false,
  "use_cache_quantization": false,
  "use_d